In [15]:
import io
import re
import string
import tqdm
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [16]:
sentence = 'I love deep learning and text mining'
tokens = list(sentence.lower().split())
print(len(tokens))

7


In [17]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'i': 1, 'love': 2, 'deep': 3, 'learning': 4, 'and': 5, 'text': 6, 'mining': 7}


In [18]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'i', 2: 'love', 3: 'deep', 4: 'learning', 5: 'and', 6: 'text', 7: 'mining'}


In [19]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 6, 7]


In [20]:
import os

# Define the URL and file path
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
file_path = 'glove.6B.zip'

# Download the file
os.system(f'wget {url} -O {file_path}')

# Unzip the file
os.system(f'unzip {file_path}')


Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        

sh: wget: command not found



  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


0

In [21]:
word_list = []
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as file:
    for line in file:
        word = line.split()[0]
        word_list.append(word)


In [22]:
embedding_dim = 50  # dimension of the word embeddings
vocab_size = len(word_list)  # assuming word_list is defined
word_index = {word: index for index, word in enumerate(word_list)}  # assuming word_list is defined

# Create the embedding layer
embedding_layer = layers.Embedding(vocab_size, embedding_dim)

# Get the word index for "deep"
word = 'deep'
word_idx = word_index[word]

# Embed the word
embedded_word = embedding_layer(tf.constant([word_idx]))

# Print the embedded word
print(embedded_word)


tf.Tensor(
[[ 0.04662981 -0.00095054 -0.02192831  0.03346461 -0.02455703 -0.01699365
   0.0155719   0.0203704   0.00427513 -0.01294947 -0.03519137 -0.00675808
  -0.02349998 -0.02286354  0.00773432 -0.02575028 -0.00721722 -0.02094314
  -0.02748066 -0.00369291  0.00612719 -0.00023375  0.00434867 -0.00063705
  -0.03567322  0.02960384 -0.02691988  0.04315214  0.0164523   0.04246105
  -0.03203094 -0.02268632  0.03460028  0.01560796  0.00170945  0.03339574
   0.01846895  0.01906809 -0.0081032  -0.01708674  0.02899529 -0.02841201
  -0.04682405  0.00248916 -0.01212237  0.04458698 -0.00694709 -0.04113496
   0.04215902  0.02787727]], shape=(1, 50), dtype=float32)


In [23]:
embedding_dim = 50  # dimension of the word embeddings
vocab_size = len(word_list)  # assuming word_list is defined
word_index = {word: index for index, word in enumerate(word_list)}  # assuming word_list is defined

# Create the embedding layer
embedding_layer = layers.Embedding(vocab_size, embedding_dim)

# Get the word index for "deep"
word = 'text'
word_idx = word_index[word]

# Embed the word
embedded_word = embedding_layer(tf.constant([word_idx]))

# Print the embedded word
print(embedded_word)

tf.Tensor(
[[ 0.04789047 -0.03611096  0.03067328 -0.01189851 -0.02462651 -0.0353537
  -0.0336131   0.00337757 -0.02382137  0.01429733  0.02520097  0.00332152
   0.02893979  0.01474389  0.04468125  0.02049011  0.03016585  0.00103295
  -0.0031607  -0.03784917  0.02448657  0.02969531  0.02932451  0.00596938
  -0.0277768   0.03140045  0.0169558   0.03025651  0.00489876 -0.0096597
   0.00305755  0.04376546  0.00863464  0.02891247  0.03754568  0.02411992
  -0.03873118 -0.01689482  0.01721006  0.02083113  0.01975257 -0.04101632
   0.04578165  0.03468609 -0.04980191  0.02111502  0.02842546  0.01714117
  -0.04550761  0.04593566]], shape=(1, 50), dtype=float32)


In [27]:
def load_glove_embeddings(file_path):
    """
    Load GloVe embeddings from a file into a dictionary.

    Args:
    - file_path: Path to the GloVe embeddings file.

    Returns:
    - A dictionary where keys are words and values are embeddings.
    """
    embeddings_dict = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = [float(val) for val in values[1:]]
            embeddings_dict[word] = vector
    return embeddings_dict

def get_embeddings_for_sentence(sentence, embeddings_dict):
    """
    Get embeddings for each word in a sentence.

    Args:
    - sentence: A string containing multiple words.
    - embeddings_dict: A dictionary of word embeddings.

    Returns:
    - A dictionary of embeddings for the sentence's words.
    """
    sentence_embeddings = {}
    for word in sentence.split():
        # Here, we convert words to lowercase to match the GloVe's casing
        word_embedding = embeddings_dict.get(word.lower())
        if word_embedding is not None:
            sentence_embeddings[word] = word_embedding
    return sentence_embeddings

# Path to the GloVe embeddings file (adjust as necessary)
glove_file_path = 'glove.6B.50d.txt'

# Load the GloVe embeddings
embeddings_dict = load_glove_embeddings(glove_file_path)

# Sentence for which to get embeddings
sentence = "I love deep learning and text mining"

# Get embeddings for the sentence
sentence_embeddings = get_embeddings_for_sentence(sentence, embeddings_dict)

# For demonstration, let's print the embeddings for "deep" and "text"
print("Embedding for 'deep':", sentence_embeddings.get("deep"))
print("# of Dimensions for 'deep':", len(sentence_embeddings.get("deep")))
print("Embedding for 'text':", sentence_embeddings.get("text"))
print("# of Dimensions for 'text':", len(sentence_embeddings.get("text")))


Embedding for 'deep': [0.31445, 1.2024, 0.066651, -0.20096, -0.049636, 0.66882, -0.049386, 0.44174, 0.1799, -0.10196, -0.43674, 0.12076, -0.12495, 0.43378, -0.87784, 0.010281, 0.54592, -0.28928, -0.46115, -0.32058, -0.69094, 0.49733, 0.40657, -0.90062, 0.69699, -1.1536, -0.12229, 1.0657, 0.93207, 0.20439, 3.3004, 0.14223, 0.46493, 0.075359, -0.56755, 0.30769, -1.1251, -0.37871, 0.57479, -0.12629, 0.13589, 0.10633, 0.058432, 0.40321, 0.10243, 0.12004, 0.41383, 0.051987, -0.5835, -1.1159]
# of Dimensions for 'deep': 50
Embedding for 'text': [0.32615, 0.36686, -0.0074905, -0.37553, 0.66715, 0.21646, -0.19801, -1.1001, -0.42221, 0.10574, -0.31292, 0.50953, 0.55775, 0.12019, 0.31441, -0.25043, -1.0637, -1.3213, 0.87798, -0.24627, 0.27379, -0.51092, 0.49324, 0.52243, 1.1636, -0.75323, -0.48053, -0.11259, -0.54595, -0.83921, 2.9825, -1.1916, -0.51958, -0.39365, -0.1419, -0.026977, 0.66296, 0.16574, -1.1681, 0.14443, 1.6305, -0.17216, -0.17436, -0.01049, -0.17794, 0.93076, 1.0381, 0.94266, -0.